In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df=pd.read_excel('C:\\Users\\nafissaad\\Tutorial\\Fundamental analysis\\Technical analysis\\NEDL_Information_Ratio.xlsx',index_col='Date')
df=df.rename(columns=df.iloc[0])
df=df[1:]
df=df.iloc[:,0:4]
df

,SPY,BLK,SCHW,QQQ
Date,,,,
2015-12-31,183.89537,296.121826,30.819532,106.951714
2016-01-04,181.324615,289.669342,29.930412,104.695267
2016-01-05,181.631287,290.417236,29.836826,104.513603
2016-01-06,179.340149,287.112671,28.816687,103.509689
2016-01-07,175.037537,274.564056,27.309868,100.268433
...,...,...,...,...
2021-07-20,431.059998,878.840027,67.940002,358.790009
2021-07-21,434.549988,882.679993,69.559998,361.559998
2021-07-22,435.459991,866.859985,67.25,363.950012


In [3]:
returns=df.pct_change()
returns

,SPY,BLK,SCHW,QQQ
Date,,,,
2015-12-31,NaN,NaN,NaN,NaN
2016-01-04,-0.013979,-0.021790,-0.028849,-0.021098
2016-01-05,0.001691,0.002582,-0.003127,-0.001735
2016-01-06,-0.012614,-0.011379,-0.034191,-0.009606
2016-01-07,-0.023991,-0.043706,-0.052290,-0.031314
...,...,...,...,...
2021-07-20,0.014330,0.039851,0.018896,0.011616
2021-07-21,0.008096,0.004369,0.023845,0.007720
2021-07-22,0.002094,-0.017923,-0.033209,0.006610


In [4]:
matrix=pd.DataFrame(columns=[returns.columns],index=['Alpha','Beta','Alpha(annualized)','Residual Risk','Appraisal ratio'])
for i in df.columns:
    matrix.loc['Alpha',i]=np.polyfit(returns['SPY'].dropna().values,returns[i].dropna().values,1)[1]
    matrix.loc['Beta',i]=np.polyfit(returns['SPY'].dropna().values,returns[i].dropna().values,1)[0]

In [5]:
matrix

,SPY,BLK,SCHW,QQQ
Alpha,0.0,0.000056,-0.000065,0.000224
Beta,1.0,1.26816,1.256667,1.08989
Alpha(annualized),NaN,NaN,NaN,NaN
Residual Risk,NaN,NaN,NaN,NaN
Appraisal ratio,NaN,NaN,NaN,NaN


In [6]:
returns['Abnormal_BLK']=returns['BLK'].values-(matrix.loc['Alpha','BLK'][0]+matrix.loc['Beta','BLK'][0]*returns['SPY'].values)
returns['Abnormal_SCHW']=returns['SCHW'].values-(matrix.loc['Alpha','SCHW'][0]+matrix.loc['Beta','SCHW'][0]*returns['SPY'].values)
returns['Abnormal_QQQ']=returns['QQQ'].values-(matrix.loc['Alpha','QQQ'][0]+matrix.loc['Beta','QQQ'][0]*returns['SPY'].values)

In [7]:
returns

,SPY,BLK,SCHW,QQQ,Abnormal_BLK,Abnormal_SCHW,Abnormal_QQQ
Date,,,,,,,
2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-04,-0.013979,-0.021790,-0.028849,-0.021098,-0.004118,-0.011217,-0.006086
2016-01-05,0.001691,0.002582,-0.003127,-0.001735,0.000381,-0.005187,-0.003803
2016-01-06,-0.012614,-0.011379,-0.034191,-0.009606,0.004562,-0.018274,0.003918
2016-01-07,-0.023991,-0.043706,-0.052290,-0.031314,-0.013338,-0.022076,-0.005390
...,...,...,...,...,...,...,...
2021-07-20,0.014330,0.039851,0.018896,0.011616,0.021621,0.000953,-0.004226
2021-07-21,0.008096,0.004369,0.023845,0.007720,-0.005954,0.013735,-0.001328
2021-07-22,0.002094,-0.017923,-0.033209,0.006610,-0.020635,-0.035775,0.004104


In [8]:
for i in df.columns:
    matrix.loc['Alpha(annualized)',i]=((1+matrix.loc['Alpha',i][0])**252)-1
    if i=='SPY':
        None
    else:
        matrix.loc['Residual Risk',i]=(returns['Abnormal_'+str(i)].std())*np.sqrt(252)
        
matrix

,SPY,BLK,SCHW,QQQ
Alpha,0.0,0.000056,-0.000065,0.000224
Beta,1.0,1.26816,1.256667,1.08989
Alpha(annualized),0.0,0.014266,-0.01624,0.058083
Residual Risk,NaN,0.166293,0.260021,0.086005
Appraisal ratio,NaN,NaN,NaN,NaN


In [9]:
matrix.loc['Appraisal ratio',][1:]=matrix.loc['Alpha(annualized)',][1:].values/matrix.loc['Residual Risk',][1:].values

In [10]:
matrix

,SPY,BLK,SCHW,QQQ
Alpha,0.0,0.000056,-0.000065,0.000224
Beta,1.0,1.26816,1.256667,1.08989
Alpha(annualized),0.0,0.014266,-0.01624,0.058083
Residual Risk,NaN,0.166293,0.260021,0.086005
Appraisal ratio,NaN,0.085789,-0.062458,0.675353


In [11]:
#QQQ found manager delivers good alpha against per the unit  of idiosyncratic risk 